# Save and load data using HDF5 files

Some of richmol objects can be stored and loaded using HDF5 files.

We start from generating field-free rotational energies and matrix elements of dipole moment and polarizability for camphor molecule. For details, see "Rotational dynamics Quickstart":

In [1]:
from richmol.rot import solve, Solution
from richmol.rot import LabTensor
from richmol.rot import Molecule
from richmol.field import CarTens

In [2]:
camphor = Molecule()

camphor.XYZ = ("angstrom",
    "O",     -2.547204,    0.187936,   -0.213755,
    "C",     -1.382858,   -0.147379,   -0.229486,
    "C",     -0.230760,    0.488337,    0.565230,
    "C",     -0.768352,   -1.287324,   -1.044279,
    "C",     -0.563049,    1.864528,    1.124041,
    "C",      0.716269,   -1.203805,   -0.624360,
    "C",      0.929548,    0.325749,   -0.438982,
    "C",      0.080929,   -0.594841,    1.638832,
    "C",      0.791379,   -1.728570,    0.829268,
    "C",      2.305990,    0.692768,    0.129924,
    "C",      0.730586,    1.139634,   -1.733020,
    "H",     -1.449798,    1.804649,    1.756791,
    "H",     -0.781306,    2.571791,    0.321167,
    "H",      0.263569,    2.255213,    1.719313,
    "H",      1.413749,   -1.684160,   -1.316904,
    "H",     -0.928638,   -1.106018,   -2.110152,
    "H",     -1.245108,   -2.239900,   -0.799431,
    "H",      1.816886,   -1.883799,    1.170885,
    "H",      0.276292,   -2.687598,    0.915376,
    "H",     -0.817893,   -0.939327,    2.156614,
    "H",      0.738119,   -0.159990,    2.396232,
    "H",      3.085409,    0.421803,   -0.586828,
    "H",      2.371705,    1.769892,    0.297106,
    "H",      2.531884,    0.195217,    1.071909,
    "H",      0.890539,    2.201894,   -1.536852,
    "H",      1.455250,    0.830868,   -2.487875,
    "H",     -0.267696,    1.035608,   -2.160680)

camphor.dip = [1.21615, -0.30746, 0.01140]

camphor.pol = [[115.80434, -0.58739, 0.03276], \
               [-0.58739, 112.28245, 1.36146], \
               [0.03276, 1.36146, 108.47809]]

camphor.frame = "diag(pol)"

sol = solve(camphor, Jmin=0, Jmax=10)

dip = LabTensor(camphor.dip, sol)
pol = LabTensor(camphor.pol, sol)

Here is how we can store and read `Molecule` object

In [3]:
# store molecule camphor
camphor.store('camphor.h5', comment="S-Camphor structure from the Supersonic expansion FTMW spectra, Kisiel, et al., PCCP 5, 820 (2003)", replace=True)

# read molecule into camphor2
camphor2 = Molecule()
camphor2.read('camphor.h5')

# obtain rotational solution for camphor2
sol2 = solve(camphor2, Jmin=0, Jmax=10)

# compare solutions for camphor and camphor2
print("\ncompare solutions")
for J in sol.keys():
    for sym in sol[J].keys():
        for i in range(sol[J][sym].nstates):
            enr = sol[J][sym].enr[i]
            enr2 = sol2[J][sym].enr[i]
            diff = enr - enr2
            print(J, "%4s"%sym, i, "%12.6f"%enr, "%12.6f"%enr2, "%12.6f"%diff)


compare solutions
0    A 0     0.000000     0.000000     0.000000
1    A 0     0.076070     0.076070     0.000000
1    A 1     0.084863     0.084863     0.000000
1    A 2     0.087741     0.087741     0.000000
2    A 0     0.227612     0.227612     0.000000
2    A 1     0.234126     0.234126     0.000000
2    A 2     0.242759     0.242759     0.000000
2    A 3     0.269138     0.269138     0.000000
2    A 4     0.269736     0.269736     0.000000
3    A 0     0.453583     0.453583     0.000000
3    A 1     0.457678     0.457678     0.000000
3    A 2     0.474864     0.474864     0.000000
3    A 3     0.497349     0.497349     0.000000
3    A 4     0.500187     0.500187     0.000000
3    A 5     0.548850     0.548850     0.000000
3    A 6     0.548929     0.548929     0.000000
4    A 0     0.752988     0.752988     0.000000
4    A 1     0.755209     0.755209     0.000000
4    A 2     0.783441     0.783441     0.000000
4    A 3     0.801159     0.801159     0.000000
4    A 4     0.808865

Here is how we can store and read rotational solutions, produced by `solve` function

In [4]:
# store solutions sol
sol.store('camphor.h5', replace=True)

# read solutions into sol2
sol2 = Solution()
sol2.read('camphor.h5')

# compute matrix elements of laboratory-frame tensors using sol2 solutions
dip2 = LabTensor(camphor.dip, sol2)
pol2 = LabTensor(camphor.pol, sol2)

# compare tensor matrix elements obtained with sol and sol2

# full matrix representations of dip and dip2 for x, y, and z components
dip_mat = [dip.tomat(form='full', repres='dense', cart=cart) for cart in 'xyz']
dip2_mat = [dip2.tomat(form='full', repres='dense', cart=cart) for cart in 'xyz']

# full matrix representations of pol and pol2 for xx, xy,, xz, yx, ... etc components
pol_mat = [pol.tomat(form='full', repres='dense', cart=cart+cart2) for cart in 'xyz' for cart2 in 'xyz']
pol2_mat = [pol2.tomat(form='full', repres='dense', cart=cart+cart2) for cart in 'xyz' for cart2 in 'xyz']

# compute and print maximal differences
import numpy
dip_diff = [numpy.max(abs(m-m2)) for m,m2 in zip(dip_mat, dip2_mat)]
pol_diff = [numpy.max(abs(m-m2)) for m,m2 in zip(pol_mat, pol2_mat)]

print("max diffs sol-sol2 in dipole matrix elements:", dip_diff)
print("max diffs sol-sol2 in polarizability matrix elements:", pol_diff)

max diffs sol-sol2 in dipole matrix elements: [0.0, 0.0, 0.0]
max diffs sol-sol2 in polarizability matrix elements: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


... and store/read Cartesian tensor operators

In [5]:
# store dip and pol matrix elements
dip.store('camphor.h5', replace=True)
pol.store('camphor.h5', replace=True)

# read matrix elements into dip2 and pol2
dip2 = CarTens(filename='camphor.h5', name='dip')
pol2 = CarTens(filename='camphor.h5', name='pol')

# compare matrix elements, calculated and stored/read form file

# full matrix representations of dip and dip2 for x, y, and z components
dip_mat = [dip.tomat(form='full', repres='dense', cart=cart) for cart in 'xyz']
dip2_mat = [dip2.tomat(form='full', repres='dense', cart=cart) for cart in 'xyz']

# full matrix representations of pol and pol2 for xx, xy,, xz, yx, ... etc components
pol_mat = [pol.tomat(form='full', repres='dense', cart=cart+cart2) for cart in 'xyz' for cart2 in 'xyz']
pol2_mat = [pol2.tomat(form='full', repres='dense', cart=cart+cart2) for cart in 'xyz' for cart2 in 'xyz']

# compute and print maximal differences
import numpy
dip_diff = [numpy.max(abs(m-m2)) for m,m2 in zip(dip_mat, dip2_mat)]
pol_diff = [numpy.max(abs(m-m2)) for m,m2 in zip(pol_mat, pol2_mat)]

print("max diffs dip-dip2:", dip_diff)
print("max diffs pol-pol2:", pol_diff)

max diffs dip-dip2: [0.0, 0.0, 0.0]
max diffs pol-pol2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


The field-free solutions can also be transformed and stored in a tensor form

In [6]:
# convert solutions sol2 into tensor form h and store
h = LabTensor(camphor, sol2)
h.store('camphor.h5', comment="S-Camphor field-free energies (in cm^-1)", replace=True)

# read solutions into h2
h2 = CarTens(filename='camphor.h5', name='h')

# compare matrix elements, calculated and stored/read form file

mat = h.tomat(form='full', repres='dense', cart='0')
mat2 = h2.tomat(form='full', repres='dense', cart='0')
diff = numpy.max(abs(mat-mat2))
print("max diffs:", diff)


max diffs: 0.0
